In [6]:
from langchain_community.utilities import SQLDatabase

In [7]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")


In [8]:
db.dialect

'sqlite'

In [9]:
db.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [10]:
db.run("select * from Genre limit 10")

"[(1, 'Rock'), (2, 'Jazz'), (3, 'Metal'), (4, 'Alternative & Punk'), (5, 'Rock And Roll'), (6, 'Blues'), (7, 'Latin'), (8, 'Reggae'), (9, 'Pop'), (10, 'Soundtrack')]"

In [12]:
OPENAI_API_KEY = ""

In [13]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [14]:
chain = create_sql_query_chain(llm,db)

In [19]:
res = chain.invoke({"question":"give me 15 artists"})
print(db.run(res))

[('AC/DC',), ('Accept',), ('Aerosmith',), ('Alanis Morissette',), ('Alice In Chains',), ('Antônio Carlos Jobim',), ('Apocalyptica',), ('Audioslave',), ('BackBeat',), ('Billy Cobham',), ('Black Label Society',), ('Black Sabbath',), ('Body Count',), ('Bruce Dickinson',), ('Buddy Guy',)]


In [21]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [22]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool


In [24]:
exec_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm,db)
chain = write_query |  exec_query

In [25]:
chain.invoke({"question":"How many employees are there"})

'[(8,)]'

In [30]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
    Question : {question}
    SQL Query : {query}
    SQL Result :{result}
    Answer
    
"""
)
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

In [27]:
answer_prompt

PromptTemplate(input_variables=['query', 'question', 'result'], template='Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n    Question : {question}\n    SQL Query : {query}\n    SQL Result :{result}\n    Answer\n    \n')

In [28]:
answer = answer_prompt | llm | StrOutputParser()

In [31]:
chain  = (
    RunnablePassthrough.assign(query=write_query).assign(result = itemgetter('query') | execute_query ) | answer
)

In [32]:
chain.invoke({"question":"how many employees are there"})

'There are 8 employees.'

In [33]:
chain.invoke({"question":"how many artists are there"})

'There are 275 artists in the database.'

In [34]:
##SQL Agent

In [37]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [38]:
agent_executor.invoke({
    "input":"List the total sales per customes, who are in top 2"
})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigade

{'input': 'List the total sales per customes, who are in top 2',
 'output': 'The total sales per customer for the top 2 customers are as follows:\n1. Customer: Helena Holý, Total Sales: $49.62\n2. Customer: Richard Cunningham, Total Sales: $47.62'}

In [39]:
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

In [45]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.6 kB)
Using cached faiss_cpu-1.8.0-cp311-cp311-macosx_11_0_arm64.whl (3.1 MB)


In [46]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [47]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [48]:
few_shot_prompt

FewShotPromptTemplate(input_variables=['dialect', 'top_k'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x103637850>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], template='User input: {input}\nSQL query: {query}'), suffix='', prefix='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the 

In [49]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [50]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [51]:
agent = create_sql_agent(
    db= db,
    llm = llm,
    prompt = full_prompt,
    agent_type="openai-tools"
)

In [52]:
agent.invoke({"input":"how many artists are there"})

{'input': 'how many artists are there',
 'output': 'There are 275 artists in the database.'}

In [53]:
##Query Checker

In [54]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
chain = create_sql_query_chain(llm, db)

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [56]:
system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""

In [57]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human","{query}")
    ]
).partial(dialect=db.dialect)

validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query":chain} | validation_chain

In [59]:
res = full_chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)

In [63]:
print(res) 

The original query looks correct and does not contain any common mistakes. Therefore, I will reproduce the original query:

```sql
SELECT AVG(i.Total) AS Average_Invoice
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
WHERE c.Country = 'USA' 
AND c.Fax IS NULL 
AND i.InvoiceDate >= '2003-01-01' 
AND i.InvoiceDate < '2010-01-01'
```


In [64]:
db.run("""
SELECT AVG(i.Total) AS Average_Invoice
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
WHERE c.Country = 'USA' 
AND c.Fax IS NULL 
AND i.InvoiceDate >= '2003-01-01' 
AND i.InvoiceDate < '2010-01-01'
""")

'[(None,)]'